In [1]:
import numpy as np
import circle_fit as cf
import os
import cv2
%run Functions.ipynb

## 重複過半的 Pixel 才拿來用

In [2]:
def Get_ROI_Loc(cy,cx,cr):
    ## Loc 從左上到右下，y放前面（符合 image 讀取）
    x, y, r = int(round(cx)), int(round(cy)), int(round(cr))
    roi_loc = [[x-r-10, y-r-10], [x+r+10, y+r+10] ]
    return np.asarray(roi_loc)

In [3]:
def Get_ROI(image):
    ## 拿一張影像先算 Center 拿來取 ROI
    _, _, _, edge = image_processing(image)
    edge_pts = get_edge_points(edge)
    cx,cy,cr,_ = cf.least_squares_circle((edge_pts))
    
    roi_loc = Get_ROI_Loc(cy,cx,cr)
    return roi_loc

In [25]:
def Analize_Multi_Frames(roi_loc, images):
    ## 取得所有當批影像的 ROI 
    lx, ly = roi_loc[0,:]
    rx, ry = roi_loc[1,:]
    imgs_crop = [image[ly:ry, lx:rx] for image in images]
    
    
    ## 取得每張邊緣影像的
    imgs_edge = []
    for cropped in imgs_crop:
        _, _, _, edge = image_processing(cropped)
        imgs_edge.append(edge)
    
    
    ## 取得每張邊緣的點
    imgs_edge_pts = [get_edge_points(edge) for edge in imgs_edge]
    
    
    ## 產生 Map 來紀錄出現次數
    Map = np.zeros(images[0].shape)
    for img_edge_pts in imgs_edge_pts:
        for x, y in img_edge_pts:
            Map[y,x] = Map[y,x] + 1
   
    
    # 塞選出重複出現一半次數的點
    valid_pts = []
    for i, row in enumerate(Map):
        for j, val in enumerate(row):
            if(val >= 5):
                valid_pts.append([j,i])
    
    cx, cy, r, _ = cf.least_squares_circle((valid_pts))
    predicted = [cx+lx, cy+ly] 
    return predicted

## 測試前檢查

In [6]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/New/Case_1")
files = os.listdir()
files.sort()
files = np.asarray(files)
batches = np.split(files, 297)

img = cv2.imread(files[0], 0)
roi_loc = Get_ROI(img)

lx, ly = roi_loc[0,:]
rx, ry = roi_loc[1,:]
imgs_crop = img[ly:ry, lx:rx]

blur, nlm, otsu, edge = image_processing(imgs_crop)

cv2.imshow("Blur", blur)
cv2.imshow("Nlm", nlm)
cv2.imshow("Otsu", otsu)
cv2.imshow("Edge", edge)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 實際測試

In [30]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/New/Case_4")

files = os.listdir()
files.sort()
files = np.asarray(files)
batches = np.split(files, 297)

predictions = []
for batch in batches:
    images = [cv2.imread(img,0) for img in batch]
    roi_loc = Get_ROI(images[0])
    predicted = Analize_Multi_Frames(roi_loc, images)
    predictions.append(predicted)
    print("{}, {}".format(predicted[0], predicted[1]))

86.33796712905507, 72.67534812564438
86.46597107039965, 72.74301307545917
86.41055526761397, 72.71868398529531
86.30283428584302, 72.59324967493225
86.4402933348428, 72.76240266032354
86.46387177622442, 72.68137996471114
86.36012327423532, 72.78733131827344
86.29802755291234, 72.84693353885541
86.43430033236791, 72.96455115401321
86.39962077678447, 72.8153764084023
86.39272191731509, 72.87085947044669
86.31160624229372, 72.77810643049244
86.33494455845187, 72.87236300477224
86.28798262180354, 72.92795420357416
86.3706065969928, 72.72580027485205
86.2554327504046, 72.7908460978399
86.40867406944295, 72.74501856039537
86.41230636154705, 72.79199948554444
86.37558490144974, 72.83018816271775
86.42408816951868, 72.7198041566906
86.38863611655145, 72.78305308577092
86.36928979010275, 72.70001727174837
86.39322284405053, 72.7606644206661
86.45762856584906, 72.81138253439264
86.33761447613334, 72.8106272666329
86.37698451782884, 72.7535940263054
86.36209847407562, 72.70734118119789
86.3367577

86.26337096292887, 72.66104967063467
86.35214283423917, 72.74792670108233
86.26115641603263, 72.69430695901731
86.31340248786753, 72.75809808269226
86.32250724938778, 72.69242620280227
86.37822433403667, 72.71653521792638
86.30824479103617, 72.73666050290672
86.37412415978028, 72.59579724255053
86.33774028947549, 72.73123278587774
86.38496996396601, 72.6888826454107
86.40866798366952, 72.90928080120753
86.40032360764353, 72.78249837955023
86.31097260395896, 72.81520796426962
86.41058954529754, 72.70296307957318
86.30848349961971, 72.74589934977831
86.33798334215001, 72.81231866497328
86.41268988381776, 72.7537842928919
86.421381333999, 72.74895884532077
86.39440456463339, 72.70298288648851
86.3839609163263, 72.71429630120602
86.30618826698283, 72.79822173695544
86.48001745962657, 72.68676840587426
86.40629730350884, 72.69078354849415
86.31219999767802, 72.78516246382469
86.33795997394395, 72.76847846082694
86.42035247314081, 72.70010477514381
86.3093085703616, 72.66038382145393
86.3806

In [31]:
show_resoult(predictions)

-----------------------------------------------------
Statistics: 
Mean Centroid: (86.37343, 72.81116)
Variance     : (0.00238, 0.00625)
Max Length   : 0.28306, 0.40339 (pixels)


## 算機率

In [22]:
from scipy.special import comb

def Count_Probability(num_tal, num_sel, prob):
    x = prob
    y = 1 - x
    
    result = 0
    for i in range(num_sel, num_tal+1):
        num_x = i 
        num_y = num_tal - i
        result = result + comb(num_tal, i) * x**num_x * y**num_y
    
    return result

Count_Probability(20,10,0.65)

0.9468333856370541